In [1]:
from sage.all import *
from sage.all import PowerSeriesRing, PolynomialRing

pretty_print_default(True)
%display latex

In [2]:
Λ, τ, σ, μ, θ = var('Lambda','tau','sigma', 'mu', 'theta')
σ2 = σ^2

In [3]:
def get_tau_vars(d):
    diag = [var(f'tau{i}{i}') for i in range(1, d+1)]
    off  = [var(f'tau{i}{j}') for i in range(1, d+1) for j in range(i+1, d+1)]
    return diag + off

def precision_matrix(d):
    if d==2:
        C = Matrix([[3/8,1/8,0],
                    [1/8,3/8,0],
                    [0,  0,  1/8]])
    elif d==3:
        C = Matrix([[1/5, 1/15,1/15, 0,    0,    0],
                    [1/15,1/5, 1/15, 0,    0,    0],
                    [1/15,1/15,1/5,  0,    0,    0],
                    [0,   0,   0,    1/15, 0,    0],
                    [0,   0,   0,    0,    1/15, 0],
                    [0,   0,   0,    0,    0,    1/15]] )
    else:
        raise ValueError("Only d=2 or 3 supported")
    return C.inverse()

In [4]:
def delta_contrast_Zel(d):
    """
    The Zeldovich toy-model Δ(τ): one convenient default delta_func.
    """
    taus = get_tau_vars(d)
    T = Matrix(SR, d); k = 0
    for i in range(d):
        T[i,i] = taus[k]; k += 1
    for i in range(d):
        for j in range(i+1,d):
            T[i,j] = T[j,i] = taus[k]; k += 1
    return 1/abs(((identity_matrix(SR,d) - T).det())) - 1

# Ellipsoidal collapse in terms of τ‐variables (Eq. 5.3):
def delta_contrast_ellipsoidal(d, nu_param = var('nu')):
    """
    Ellipsoidal‐collapse density contrast Δ(τ) ≡ ρ(τ) − 1
      - d         : dimension (e.g. 3)
      - nu_param  : ellipticity exponent ν
    """

    taus = get_tau_vars(d)
    T = Matrix(SR, d); k = 0
    for i in range(d):
        T[i,i] = taus[k]; k += 1
    for i in range(d):
        for j in range(i+1,d):
            T[i,j] = T[j,i] = taus[k]; k += 1
    
    # 2) linear contrast δ = sum_i tau_ii
    delta_lin = T.trace()
    
    # 3) build the total density ρ(τ) and subtract 1 to get Δ
    rho = (1 - delta_lin/d)^d / (1 - delta_lin/nu_param)^nu_param / ((identity_matrix(SR,d) - T).det())

    return rho - 1

def theta_Zel(d):
    """
    Zeldovich velocity-divergence map:
      theta = - Tr[(I - T)^(-1) T]
    (f=1 assumed; multiply by f(eta) if needed).
    """
    taus = get_tau_vars(d)
    T = Matrix(SR, d)
    k = 0
    for i in range(d):
        T[i,i] = taus[k]; k += 1
    for i in range(d):
        for j in range(i+1, d):
            T[i,j] = T[j,i] = taus[k]; k += 1

    IminT_inv = (identity_matrix(SR, d) - T).inverse()
    return -(IminT_inv * T).trace().factor()

In [5]:
def theta_Zel_spherical(d):
    theta = theta_Zel(d)
    taus = get_tau_vars(d)
    subs = {v: τ for v in taus[:d]}
    subs.update({v: 0 for v in taus[d:]})
    return theta.subs(subs).factor()

show(theta_Zel_spherical(2))
show(theta_Zel_spherical(3))

2*tau/(tau - 1)

3*tau/(tau - 1)

In [6]:
# 2) Joint action in the full tau-space
def action_joint(d, Λ, μ, nu=None):
    if nu is None:
        nu = d
    taus = get_tau_vars(d)
    Δ    = delta_contrast_ellipsoidal(d, nu)
    θ    = theta_Zel(d)
    P    = precision_matrix(d)
    quad = sum(taus[i] * sum(P[i,j]*taus[j] for j in range(len(taus)))
               for i in range(len(taus)))
    return Λ*Δ + θ*μ - (1/2)*quad

# 3) Spherical reduction: τ_{ij} = τ * δ_{ij} -> single var
def action_spherical_joint(d, Λ, μ, nu=None):
    # substitute τ_ii=τ, τ_ij≠i=0 into joint action
    S_full = action_joint(d, Λ, μ, nu)
    taus = get_tau_vars(d)
    subs = {v: τ for v in taus[:d]}
    subs.update({v: 0 for v in taus[d:]})
    return S_full.subs(subs)


In [7]:
def hessian_joint_full(d, Λ, μ, nu=None):
    if nu is None:
        nu = d
    taus = get_tau_vars(d)
    S    = action_joint(d, Λ, μ, nu)
    n    = len(taus)
    H    = Matrix(SR, n, n)
    for i in range(n):
        for j in range(n):
            H[i,j] = diff(S, taus[i], taus[j])
    return H
def hessian_spherical(d, Λ, μ, nu=None):
    if nu is None:
        nu = d
    Hf   = hessian_joint_full(d, Λ, μ, nu)
    taus = get_tau_vars(d)
    subs = {v:τ for v in taus[:d]}
    subs.update({v:0 for v in taus[d:]})
    return Hf.subs(subs).apply_map(lambda x: x.factor())


In [8]:
show(action_joint(3, Λ, μ))
show(action_spherical_joint(3, Λ, μ))

-3/4*(4*tau11 - tau22 - tau33)*tau11 - 15/2*tau12^2 - 15/2*tau13^2 + 3/4*(tau11 - 4*tau22 + tau33)*tau22 - 15/2*tau23^2 + 3/4*(tau11 + tau22 - 4*tau33)*tau33 + Lambda*(1/((tau23^2 - (tau22 - 1)*(tau33 - 1))*(tau11 - 1) - (tau13*tau23 - tau12*(tau33 - 1))*tau12 + (tau13*(tau22 - 1) - tau12*tau23)*tau13) - 1) + (3*tau13^2*tau22 - 6*tau12*tau13*tau23 + 3*tau11*tau23^2 + 3*tau12^2*tau33 - 3*tau11*tau22*tau33 - 2*tau12^2 - 2*tau13^2 + 2*tau11*tau22 - 2*tau23^2 + 2*tau11*tau33 + 2*tau22*tau33 - tau11 - tau22 - tau33)*mu/(tau13^2*tau22 - 2*tau12*tau13*tau23 + tau11*tau23^2 + tau12^2*tau33 - tau11*tau22*tau33 - tau12^2 - tau13^2 + tau11*tau22 - tau23^2 + tau11*tau33 + tau22*tau33 - tau11 - tau22 - tau33 + 1)

-9/2*tau^2 - Lambda*(1/(tau - 1)^3 + 1) + 3*(tau^3 - 2*tau^2 + tau)*mu/(tau^3 - 3*tau^2 + 3*tau - 1)

In [9]:
show(hessian_joint_full(2, Λ, μ))
show(hessian_spherical(2, Λ=(2*τ*(1-τ)**3), μ=0))

[                                                                                                                                                                                                                                                                                                -2*mu*(2*tau22 - 1)*(tau22 - 1)/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^2 + 2*(2*tau12^2 - 2*tau11*tau22 + tau11 + tau22)*mu*(tau22 - 1)^2/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^3 - 2*Lambda*(tau22 - 1)^2/(tau12^2 - (tau11 - 1)*(tau22 - 1))^3 - 3 -mu*(tau11 - 1)*(2*tau22 - 1)/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^2 - mu*(2*tau11 - 1)*(tau22 - 1)/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^2 + 2*(2*tau12^2 - 2*tau11*tau22 + tau11 + tau22)*mu*(tau11 - 1)*(tau22 - 1)/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^3 + (2*tau12^2 - 2*tau11*tau22 + tau11 + tau22)*mu/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^2 - 2*mu/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1) - 2*Lambda*(tau11 - 1)*(tau22 - 1)/(tau12^2 - (tau11 - 1)*(tau22 - 1))^3 - Lambda/(tau12^2 - (tau11 - 1)*(tau22 - 1))^2 + 1                                                                                                                                                                                                                             2*mu*tau12*(2*tau22 - 1)/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^2 - 4*(2*tau12^2 - 2*tau11*tau22 + tau11 + tau22)*mu*tau12*(tau22 - 1)/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^3 + 4*mu*tau12*(tau22 - 1)/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^2 + 4*Lambda*tau12*(tau22 - 1)/(tau12^2 - (tau11 - 1)*(tau22 - 1))^3]
[-mu*(tau11 - 1)*(2*tau22 - 1)/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^2 - mu*(2*tau11 - 1)*(tau22 - 1)/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^2 + 2*(2*tau12^2 - 2*tau11*tau22 + tau11 + tau22)*mu*(tau11 - 1)*(tau22 - 1)/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^3 + (2*tau12^2 - 2*tau11*tau22 + tau11 + tau22)*mu/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^2 - 2*mu/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1) - 2*Lambda*(tau11 - 1)*(tau22 - 1)/(tau12^2 - (tau11 - 1)*(tau22 - 1))^3 - Lambda/(tau12^2 - (tau11 - 1)*(tau22 - 1))^2 + 1                                                                                                                                                                                                                                                                                                 -2*mu*(2*tau11 - 1)*(tau11 - 1)/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^2 + 2*(2*tau12^2 - 2*tau11*tau22 + tau11 + tau22)*mu*(tau11 - 1)^2/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^3 - 2*Lambda*(tau11 - 1)^2/(tau12^2 - (tau11 - 1)*(tau22 - 1))^3 - 3                                                                                                                                                                                                                             2*mu*(2*tau11 - 1)*tau12/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^2 - 4*(2*tau12^2 - 2*tau11*tau22 + tau11 + tau22)*mu*(tau11 - 1)*tau12/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^3 + 4*mu*(tau11 - 1)*tau12/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^2 + 4*Lambda*(tau11 - 1)*tau12/(tau12^2 - (tau11 - 1)*(tau22 - 1))^3]
[                                                                                                                                                                                                                            2*mu*tau12*(2*tau22 - 1)/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^2 - 4*(2*tau12^2 - 2*tau11*tau22 + tau11 + tau22)*mu*tau12*(tau22 - 1)/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^3 + 4*mu*tau12*(tau22 - 1)/(tau12^2 - tau11*tau22 + tau11 + tau22 - 1)^2 + 4*Lambda*tau12*(tau22 - 1)/(tau12^2 - (tau11 - 1)*(tau22 - 1))^3                                                                                                                                                                                            

[  -(7*tau - 3)/(tau - 1)     -(tau + 1)/(tau - 1)                        0]
[    -(tau + 1)/(tau - 1)   -(7*tau - 3)/(tau - 1)                        0]
[                       0                        0 -4*(3*tau - 2)/(tau - 1)]

In [10]:
def saddle_equation(d, nu=None):
    # 1) Build the 1D action under spherical ansatz
    S_tau = action_spherical_joint(d, Λ, μ, nu)
    # 2) Compute derivative w.r.t. tau
    eq = diff(S_tau, τ).numerator().factor()
    # only normalize by the Λ-coefficient when it’s nonzero
    #coeff_L = eq.coefficient(Λ)
    #if coeff_L != 0:
        #eq /= coeff_L
    return eq

In [11]:
show("Saddle eq (d=2):", saddle_equation(2).factor() == 0)
show("Saddle eq (d=3):", saddle_equation(3).factor() == 0)

'Saddle eq (d=2):' -4*tau^4 + 12*tau^3 - 2*mu*tau - 12*tau^2 - 2*Lambda + 2*mu + 4*tau == 0

'Saddle eq (d=3):' -9*tau^5 + 36*tau^4 - 3*mu*tau^2 - 54*tau^3 + 6*mu*tau + 36*tau^2 + 3*Lambda - 3*mu - 9*tau == 0

In [12]:
def tau_series(d, total_order=4, nu=None):
    """
    Return τ(μ,Λ) to the requested total order
    using Hensel/Newton lifting inside a power-series ring.
    """    
    if nu is None:
        nu = d

    # total-degree default_prec must exceed the order we want
    R.<mu, Lambda> = PowerSeriesRing(QQ, default_prec=total_order+2)
    PR.<tau>       = PolynomialRing(R)
    # build polynomial version of the saddle equation
    P_poly   = PR(saddle_equation(d, nu=nu).expand())
    dP_poly  = P_poly.derivative()

    # initial guess: constant term killing P at O(mu, Lambda)
    root = PR((Lambda - mu)/d)

    # Newton–Hensel lift in the polynomial ring over R
    for k in range(1, total_order+1):
        valP  = P_poly(root)
        valdP = dP_poly(root)
        # Convert to power series and truncate before division
        valP_trunc = R(valP).truncate(k+1)
        valdP_trunc = R(valdP).truncate(k+1)
        correction = valP_trunc / valdP_trunc
        root = root - PR(correction)
        root = root.truncate(k+1)

    return R(root).truncate(total_order+1).add_bigoh(total_order+1)

# examples for both dimensions
show("τ* (d=2):", tau_series(2))
show("τ* (d=3):", tau_series(3))


'τ* (d=2):' -1/2*mu + 1/2*Lambda + 1/2*mu^2 - 5/4*mu*Lambda + 3/4*Lambda^2 - 7/8*mu^3 + 7/2*mu^2*Lambda - 9/2*mu*Lambda^2 + 15/8*Lambda^3 + 15/8*mu^4 - 165/16*mu^3*Lambda + 165/8*mu^2*Lambda^2 - 143/8*mu*Lambda^3 + 91/16*Lambda^4 + O(mu, Lambda)^5

'τ* (d=3):' -1/3*mu + 1/3*Lambda + 2/9*mu^2 - 2/3*mu*Lambda + 4/9*Lambda^2 - 7/27*mu^3 + 4/3*mu^2*Lambda - 55/27*mu*Lambda^2 + 26/27*Lambda^3 + 10/27*mu^4 - 220/81*mu^3*Lambda + 182/27*mu^2*Lambda^2 - 560/81*mu*Lambda^3 + 68/27*Lambda^4 + O(mu, Lambda)^5

In [13]:
from sage.all import QQ, PowerSeriesRing, var
from sage.misc.sage_eval import sage_eval

def joint_CGF_series(d, total_order=4, nu=None):
    """
    Generic LO joint CGF S(μ,Λ) as a multivariate power series.
    Relies on your existing saddle_equation() and action_spherical_joint().
    """
    if nu is None:
        nu = d

    # ─── 0) set up the *symbolic* variables ──────────────────────────────
    mu_sym, Lambda_sym, tau_sym = var('mu', 'Lambda', 'tau')

    # ─── 1) build the *power*-series ring R and its τ–polynomial ring S ─
    R.<mu,Lambda> = PowerSeriesRing(QQ, default_prec=total_order+2)
    S.<tau>   = R[]

    # ─── helper to re-parse a symbolic expr into *this* ring ────────────
    def _to_ring(expr_sym):
        return sage_eval(str(expr_sym),
                         locals={'mu':mu,   'Lambda':Lambda,
                                 'tau':tau})

    # ─── 2) get your symbolic quintic and turn it into S───────────────
    P_sym = saddle_equation(d, nu=nu)            # returns Expr in mu_sym, Lambda_sym, tau_sym
    P     = _to_ring(P_sym)                      # now P ∈ S

    # ─── 3) Newton/Hensel lift the τ≈0 root inside S───────────────────
    tau_star = (Lambda - mu)/3
    dP       = P.derivative(tau)
    for k in range(1, total_order+1):
        r = P(tau=tau_star).truncate(k+1)
        j = dP(tau=tau_star).truncate(k+1)
        tau_star -= r/j
        tau_star  = tau_star.truncate(k+1)
    tau_star = tau_star.truncate(total_order+1).add_bigoh(total_order+1)
    show("τ★ (d={}, nu={}):".format(d, nu), tau_star)

    # ─── 4) pull in your symbolic joint‐action and convert it──────────
    A_sym = action_spherical_joint(d, Lambda_sym, mu_sym, nu=nu)
    A     = _to_ring(A_sym)                      # A ∈ S

    # ─── 5) evaluate the polynomial in τ at τ★ and truncate─x──────────
    return A(tau=tau_star) \
             .truncate(total_order+1) \
             .add_bigoh(total_order+1)

show("Joint CGF (d=2):", joint_CGF_series(2, total_order=4))
show("Joint CGF (d=3):", joint_CGF_series(3, total_order=4))

'τ★ (d=2, nu=2):' -1/2*mu + 1/2*Lambda + 1/2*mu^2 - 5/4*mu*Lambda + 3/4*Lambda^2 - 7/8*mu^3 + 7/2*mu^2*Lambda - 9/2*mu*Lambda^2 + 15/8*Lambda^3 + 15/8*mu^4 - 165/16*mu^3*Lambda + 165/8*mu^2*Lambda^2 - 143/8*mu*Lambda^3 + 91/16*Lambda^4 + O(mu, Lambda)^5

'Joint CGF (d=2):' 1/2*mu^2 - mu*Lambda + 1/2*Lambda^2 - 1/2*mu^3 + 7/4*mu^2*Lambda - 2*mu*Lambda^2 + 3/4*Lambda^3 + 3/4*mu^4 - 15/4*mu^3*Lambda + 55/8*mu^2*Lambda^2 - 11/2*mu*Lambda^3 + 13/8*Lambda^4 + O(mu, Lambda)^5

'τ★ (d=3, nu=3):' -1/3*mu + 1/3*Lambda + 2/9*mu^2 - 2/3*mu*Lambda + 4/9*Lambda^2 - 7/27*mu^3 + 4/3*mu^2*Lambda - 55/27*mu*Lambda^2 + 26/27*Lambda^3 + 10/27*mu^4 - 220/81*mu^3*Lambda + 182/27*mu^2*Lambda^2 - 560/81*mu*Lambda^3 + 68/27*Lambda^4 + O(mu, Lambda)^5

'Joint CGF (d=3):' 1/2*mu^2 - mu*Lambda + 1/2*Lambda^2 - 1/3*mu^3 + 4/3*mu^2*Lambda - 5/3*mu*Lambda^2 + 2/3*Lambda^3 + 1/3*mu^4 - 55/27*mu^3*Lambda + 13/3*mu^2*Lambda^2 - 35/9*mu*Lambda^3 + 34/27*Lambda^4 + O(mu, Lambda)^5

In [14]:
def one_loop_correction_joint(d, Λ, μ, nu=None):
    if nu is None:
        nu = d
    Hj = -hessian_spherical(d, Λ, μ, nu)
    return - (1/2) * log( (Hj.det()/precision_matrix(d).det()).factor() )

def CGF_NLO_joint(d, Λ, μ, σ2, nu=None, total_order=4):
    if nu is None:
        nu = d
    φ0 = joint_CGF_series(d, total_order, nu)
    F1 = one_loop_correction_joint(d, Λ, μ, nu)
    return (φ0 + σ2*F1).factor()


In [15]:
show(one_loop_correction_joint(2, Λ, μ))
show(one_loop_correction_joint(2, Λ=(2*τ*(1-τ)**3), μ=0))

-1/2*log(1/32*(4*tau^4 - 16*tau^3 - 2*mu*tau + 24*tau^2 - Lambda + 2*mu - 16*tau + 4)^2*(2*tau^4 - 8*tau^3 - 2*mu*tau + 12*tau^2 - 3*Lambda + 2*mu - 8*tau + 2)/(tau - 1)^12)

-1/2*log(1/4*(4*tau - 1)*(3*tau - 2)^2/(tau - 1)^3)

In [16]:
from sage.all import QQ, PolynomialRing, PowerSeriesRing, Integer, var, Matrix
from sage.misc.sage_eval import sage_eval

# ─── series_log as before ─────────────────────────────────────────────────
def series_log(F, N):
    c0 = F.constant_coefficient()
    δ  = F/c0 - 1
    L  = δ.parent()(0)
    for k in range(1, N+1):
        L += ((-1)**(k+1) * δ**k) / k
    return L.truncate(N+1)

# ─── joint_CGF_series_with_NLO using full spherical Hessian ───────────────
def joint_CGF_series_with_NLO(d, total_order=4, nu=None, sigma=σ):
    """
    LO+NLO joint CGF S(μ,Λ) with the one-loop term built from
    det[H_spherical(τ★)] rather than ∂²A/∂τ² alone.
    """
    # default spectral slope
    if nu is None:
        nu = d

    # symbolic placeholders
    mu_sym, Lambda_sym, tau_sym, nu_sym, sigma_sym = var('mu','Lambda','tau','nu','sigma')

    # decide base ring so sigma can be symbolic
    if sigma is None or isinstance(sigma_sym, (int, Integer)):
        Base     = QQ
        s_param  = Integer(1) if sigma is None else Integer(sigma)
    else:
        Base     = PolynomialRing(QQ, 'sigma')
        s_param  = Base.gen()

    # build the power-series ring and tau-polynomial ring
    R.<mu,Lambda> = PowerSeriesRing(Base, default_prec=total_order+2)
    S.<tau>       = R[]

    # helper to re-parse symbolic → R[τ]
    env = {'mu':mu, 'Lambda':Lambda, 'tau':tau, 'nu':Integer(nu), 'sigma':s_param}
    def _to_ring(expr_sym):
        return sage_eval(str(expr_sym), locals=env)

    # saddle eq. & Newton/Hensel for τ★
    P       = _to_ring(saddle_equation(d, nu=nu_sym))
    tau_star= (Lambda - mu)/Integer(3)
    dP      = P.derivative(tau)
    for k in range(1, total_order+1):
        r = P(tau=tau_star).truncate(k+1)
        j = dP(tau=tau_star).truncate(k+1)
        tau_star -= r/j
        tau_star  = tau_star.truncate(k+1)
    tau_star = tau_star.truncate(total_order+1)

    # 7) leading-order action
    A    = _to_ring(action_spherical_joint(d, Lambda_sym, mu_sym, nu=nu_sym))
    S_LO = A(tau=tau_star).truncate(total_order+1).add_bigoh(total_order+1)

    # 8) full spherical Hessian
    Hf_sym    = hessian_spherical(d, Lambda_sym, mu_sym, nu=nu_sym)
    n         = Hf_sym.nrows()
    H_entries = [[_to_ring(Hf_sym[i,j]) for j in range(n)] for i in range(n)]
    H_sph     = Matrix(H_entries)

    # 9) evaluate at τ★
    H_star = H_sph.apply_map(lambda f: f(tau=tau_star))

    # 10) one-loop = –½ σ² log(det H_star / detΣ⁻¹)
    detΣinv  = precision_matrix(d).det()
    rat      = H_star.det() / detΣinv
    one_loop = (-Integer(1)/Integer(2)) * series_log(rat, total_order)

    # combine LO + σ²·one_loop
    return (S_LO + s_param**2 * one_loop) \
               .truncate(total_order+1) \
               .add_bigoh(total_order+1)


In [17]:
# ─── Examples ────────────────────────────────────────────────────────────
show("CGF(d=2):",joint_CGF_series_with_NLO(d=2, total_order=4))
show("CGF(d=3):",joint_CGF_series_with_NLO(d=3, total_order=4))

'CGF(d=2):' (-sigma^2)*mu + (sigma^2)*Lambda + (15*sigma^2 + 4)/8*mu^2 + (-35*sigma^2 - 8)/8*mu*Lambda + (83*sigma^2 + 16)/32*Lambda^2 + (-26*sigma^2 - 3)/6*mu^3 + (65*sigma^2 + 7)/4*mu^2*Lambda + (-651*sigma^2 - 64)/32*mu*Lambda^2 + (1621*sigma^2 + 144)/192*Lambda^3 + (707*sigma^2 + 48)/64*mu^4 + (-1839*sigma^2 - 120)/32*mu^3*Lambda + (14293*sigma^2 + 880)/128*mu^2*Lambda^2 + (-12247*sigma^2 - 704)/128*mu*Lambda^3 + (31177*sigma^2 + 1664)/1024*Lambda^4 + O(mu, Lambda)^5

'CGF(d=3):' (-sigma^2)*mu + (sigma^2)*Lambda + (12*sigma^2 + 5)/10*mu^2 + (-16*sigma^2 - 5)/5*mu*Lambda + (64*sigma^2 + 15)/30*Lambda^2 + (-1214*sigma^2 - 225)/675*mu^3 + (619*sigma^2 + 100)/75*mu^2*Lambda + (-2756*sigma^2 - 375)/225*mu*Lambda^2 + (3943*sigma^2 + 450)/675*Lambda^3 + (3368*sigma^2 + 375)/1125*mu^4 + (-67058*sigma^2 - 6875)/3375*mu^3*Lambda + (160316*sigma^2 + 14625)/3375*mu^2*Lambda^2 + (-163562*sigma^2 - 13125)/3375*mu*Lambda^3 + (60284*sigma^2 + 4250)/3375*Lambda^4 + O(mu, Lambda)^5

In [18]:
def driver_joint_cumulants(d, max_order, nu=None, sigma=σ):
    """
    Compute the joint cumulants κ_{i,j} up to total order max_order
    in (μ,Λ), carrying along the 1-loop σ² dependence.

    Returns
    -------
    cum : dict[(i,j) -> ring element]
      κ_{i,j} = (∂^i_μ ∂^j_Λ S)(0,0) · i!·j! for i+j ≤ max_order
    """
    from sage.all import factorial, var

    # 1) build the full LO+NLO CGF series S(μ,Λ)
    mu, Lambda, sigma_sym = var('mu', 'Lambda', 'sigma')
    S = joint_CGF_series_with_NLO(
            d,
            total_order=max_order,
            nu=nu,
            sigma=sigma_sym
        )

    # 2) extract the monomial→coefficient map
    #    keys are (i,j), values are the coefficient of μ^i Λ^j
    monos = S.monomial_coefficients()

    # 3) form cumulants = coeff * i! * j!
    cum = {}
    for (i,j), c in monos.items():
        if i + j <= max_order:
            cum[(i,j)] = c * factorial(i) * factorial(j)
    return cum


In [19]:
# define your σ symbol
sigma = var('sigma')

# compute 2D joint cumulants up to total order 4
cumulants = driver_joint_cumulants(d=3, max_order=4, nu=3, sigma=sigma)

show("κ_{1,0} =", cumulants[(1,0)])   # linear-in-μ term
show("κ_{0,1} =", cumulants[(0,1)])   # linear-in-Λ term
show("κ_{2,0} =", cumulants[(2,0)])   # variance of δ
show("κ_{1,1} =", cumulants[(1,1)])   # mixed cumulant
show("κ_{0,2} =", cumulants[(0,2)])   # variance of θ


'κ_{1,0} =' -sigma^2

'κ_{0,1} =' sigma^2

'κ_{2,0} =' 12/5*sigma^2 + 1

'κ_{1,1} =' -16/5*sigma^2 - 1

'κ_{0,2} =' 64/15*sigma^2 + 1

In [20]:
# Cell is ready for code